[의문]
- loss 무엇?

In [32]:
# 모듈 불러오기
from collections import Counter

import nltk
nltk.download('punkt')
from nltk import word_tokenize

from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


import numpy as np
import string
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"

# _1_. 데이터

* 데이터 생김새

```
Alice\'s Adventures in Wonderland\n\n                ALICE\'S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3.0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought Alice `without pictures or conversation?\'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid),
```



In [8]:
# 데이터 로드
with open(f"{data_path}/alice_in_wonderland.txt", "r") as f:
    content = f.read()

# _2_. 전처리

## 1) 구둣점 제거

* 구둣점 제거

    - `[" " if char in string.punctuation else char for char in content]` : 한 글자씩
    ```
    'r',
 'a',
 'n',
 ' ',
 'c',
 'l',
 'o'
 ```

    - `"".join([" " if char in string.punctuation else char for char in content])` : 띄어쓰기 되어 있음
    ```
    Alice s Adventures in Wonderland\n\n                ALICE S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3 0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank  and of having nothing to do   once or twice she had\npeeped into the book her sister was reading  but it had n
    ```
    - `"".join([" " if char in string.punctuation else char for char in content].split()` : 단어별로 구분
    - 최종
    ```
    Alice s Adventures in Wonderland ALICE S ADVENTURES IN WONDERLAND Lewis Carroll THE MILLENNIUM FULCRUM EDITION 3 0 CHAPTER I Down the Rabbit Hole Alice was beginning to get very tired of sitting by her sister on the bank and of having nothin
    ```

## 2) 토큰화

## 3) 소문자화

## 4) 철자 2개 이상


In [9]:
# 구둣점 제거
text = " ".join("".join([" " if char in string.punctuation else char for char in content]).split())
text

'Alice s Adventures in Wonderland ALICE S ADVENTURES IN WONDERLAND Lewis Carroll THE MILLENNIUM FULCRUM EDITION 3 0 CHAPTER I Down the Rabbit Hole Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures or conversation So she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a White Rabbit with pink eyes ran close by her There was nothing so VERY remarkable in that nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself Oh dear Oh dear I shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it al

In [10]:
# 토큰화
tokens = word_tokenize(text)
tokens = [word.lower() for word in tokens if len(word) >= 2]
tokens

['alice',
 'adventures',
 'in',
 'wonderland',
 'alice',
 'adventures',
 'in',
 'wonderland',
 'lewis',
 'carroll',
 'the',
 'millennium',
 'fulcrum',
 'edition',
 'chapter',
 'down',
 'the',
 'rabbit',
 'hole',
 'alice',
 'was',
 'beginning',
 'to',
 'get',
 'very',
 'tired',
 'of',
 'sitting',
 'by',
 'her',
 'sister',
 'on',
 'the',
 'bank',
 'and',
 'of',
 'having',
 'nothing',
 'to',
 'do',
 'once',
 'or',
 'twice',
 'she',
 'had',
 'peeped',
 'into',
 'the',
 'book',
 'her',
 'sister',
 'was',
 'reading',
 'but',
 'it',
 'had',
 'no',
 'pictures',
 'or',
 'conversations',
 'in',
 'it',
 'and',
 'what',
 'is',
 'the',
 'use',
 'of',
 'book',
 'thought',
 'alice',
 'without',
 'pictures',
 'or',
 'conversation',
 'so',
 'she',
 'was',
 'considering',
 'in',
 'her',
 'own',
 'mind',
 'as',
 'well',
 'as',
 'she',
 'could',
 'for',
 'the',
 'hot',
 'day',
 'made',
 'her',
 'feel',
 'very',
 'sleepy',
 'and',
 'stupid',
 'whether',
 'the',
 'pleasure',
 'of',
 'making',
 'daisy',
 'ch

# _3_. N그램

In [12]:
N = int(input('N 개수 설정: '))

quads = list(nltk.ngrams(tokens, N))
quads

N 개수 설정: 5


[('alice', 'adventures', 'in', 'wonderland', 'alice'),
 ('adventures', 'in', 'wonderland', 'alice', 'adventures'),
 ('in', 'wonderland', 'alice', 'adventures', 'in'),
 ('wonderland', 'alice', 'adventures', 'in', 'wonderland'),
 ('alice', 'adventures', 'in', 'wonderland', 'lewis'),
 ('adventures', 'in', 'wonderland', 'lewis', 'carroll'),
 ('in', 'wonderland', 'lewis', 'carroll', 'the'),
 ('wonderland', 'lewis', 'carroll', 'the', 'millennium'),
 ('lewis', 'carroll', 'the', 'millennium', 'fulcrum'),
 ('carroll', 'the', 'millennium', 'fulcrum', 'edition'),
 ('the', 'millennium', 'fulcrum', 'edition', 'chapter'),
 ('millennium', 'fulcrum', 'edition', 'chapter', 'down'),
 ('fulcrum', 'edition', 'chapter', 'down', 'the'),
 ('edition', 'chapter', 'down', 'the', 'rabbit'),
 ('chapter', 'down', 'the', 'rabbit', 'hole'),
 ('down', 'the', 'rabbit', 'hole', 'alice'),
 ('the', 'rabbit', 'hole', 'alice', 'was'),
 ('rabbit', 'hole', 'alice', 'was', 'beginning'),
 ('hole', 'alice', 'was', 'beginning', 

In [13]:
# N 그램 학습 데이터 형태로 변환
new_text = []
for q in quads:
    temp = " ".join(q)
    new_text.append(temp)

new_text

['alice adventures in wonderland alice',
 'adventures in wonderland alice adventures',
 'in wonderland alice adventures in',
 'wonderland alice adventures in wonderland',
 'alice adventures in wonderland lewis',
 'adventures in wonderland lewis carroll',
 'in wonderland lewis carroll the',
 'wonderland lewis carroll the millennium',
 'lewis carroll the millennium fulcrum',
 'carroll the millennium fulcrum edition',
 'the millennium fulcrum edition chapter',
 'millennium fulcrum edition chapter down',
 'fulcrum edition chapter down the',
 'edition chapter down the rabbit',
 'chapter down the rabbit hole',
 'down the rabbit hole alice',
 'the rabbit hole alice was',
 'rabbit hole alice was beginning',
 'hole alice was beginning to',
 'alice was beginning to get',
 'was beginning to get very',
 'beginning to get very tired',
 'to get very tired of',
 'get very tired of sitting',
 'very tired of sitting by',
 'tired of sitting by her',
 'of sitting by her sister',
 'sitting by her sister o

# _4_. 벡터화: CountVectorizer 이용

- count vectorizer로 벡터화하면 가각ㄱ의 단어에서 위치가 어디있는지 나온다.
- X matrix에는 2개 있어야 하고, y matrix에는 1개 있어야 한다.

In [14]:
# countvectorize

vectorizer = CountVectorizer()

X_trigram = []
y_trigram = []

for text in new_text:
    x_str = " ".join(text.split()[0:N-2]) # 앞의 N-1 단어
    y_str = " ".join(text.split()[N-2:]) # 뒤에 나와야 할 단어
    X_trigram.append(x_str)
    y_trigram.append(y_str)

In [16]:
# countvertorizer 이용 벡터화
X_trigram_mat = vectorizer.fit_transform(X_trigram).todense()
print(id(vectorizer))
print(vectorizer)
print(vectorizer.vocabulary_)
print()
y_trigram_mat = vectorizer.fit_transform(y_trigram).todense()
print(id(vectorizer))
print(vectorizer)
print(vectorizer.vocabulary_)

140015565704328
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
{'alice': 51, 'adventures': 27, 'in': 1088, 'wonderland': 2510, 'lewis': 1242, 'carroll': 303, 'the': 2205, 'millennium': 1363, 'fulcrum': 869, 'edition': 641, 'chapter': 334, 'down': 591, 'rabbit': 1719, 'hole': 1034, 'was': 2430, 'beginning': 183, 'to': 2261, 'get': 895, 'very': 2398, 'tired': 2258, 'of': 1478, 'sitting': 1988, 'by': 278, 'her': 1010, 'sister': 1984, 'on': 1491, 'bank': 147, 'and': 70, 'having': 987, 'nothing': 1461, 'do': 570, 'once': 1492, 'or': 1506, 'twice': 2331, 'she': 1925, 'had': 955, 'peeped': 1569, 'into': 1111, 'book': 225, 'reading'

# _5_. 어휘집
- countvectorizer의 vocabulary_ 메소드 쓰면 워드 투 인덱스 어휘집이 나온다.



In [17]:
# 어휘집
word2idx = vectorizer.vocabulary_
idx2word = {v:k for k, v in word2idx.items()} 

# _6_. 모델 구성

In [18]:
# 데이터 준비
X = np.array(X_trigram_mat)
y = np.array(y_trigram_mat)

# train, test 분리
X_train, X_test, y_train, y_test, X_train_tg, X_test_tg = train_test_split(X, y, X_trigram,
                                                                            test_size=0.3,
                                                                            random_state=42)
print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")
print(f"Train Trigram: {len(X_train_tg)}, {len(X_test_tg)}")

Train: (17945, 2559), (17945, 2559)
Test: (7692, 2559), (7692, 2559)
Train Trigram: 17945, 7692


In [19]:
# 모델 파라미터 설정
EPOCHS = int(input('학습 에폭 수 설정: '))
BATCH = int(input('배치 사이즈 설정: '))

학습 에폭 수 설정: 300
배치 사이즈 설정: 64


In [34]:
# FFN 네트워크 구성
X_input = Input(shape=(X_train.shape[1], ), name='input')
X_dense_1 = Dense(1000, activation='relu', name='1st_hidden')(X_input)
X_dense_1 = Dropout(0.5, name='1st_dout')(X_dense_1)
X_dense_2 = Dense(800, activation='relu', name='2nd_hidden')(X_dense_1)
X_dense_3 = Dense(1000, activation='relu', name='3rd_hidden')(X_dense_2)
X_dense_3 = Dropout(0.5, name='2nd_dout')(X_dense_3)
y_output = Dense(y_train.shape[1], activation='sigmoid', name='output')(X_dense_3)

In [38]:
K.clear_session()

# 모델 구성
model = Model(X_input, y_output)
model.compile(optimizer='adam', loss='binary_crossentropy')
print("======= 모델 전체 구조 ======")
print(model.summary())

======= 모델 전체 구조 ======
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2559)]            0         
_________________________________________________________________
1st_hidden (Dense)           (None, 1000)              2560000   
_________________________________________________________________
1st_dout (Dropout)           (None, 1000)              0         
_________________________________________________________________
2nd_hidden (Dense)           (None, 800)               800800    
_________________________________________________________________
3rd_hidden (Dense)           (None, 1000)              801000    
_________________________________________________________________
2nd_dout (Dropout)           (None, 1000)              0         
_________________________________________________________________
output (Dense)               (None, 2

# _7_. 학습

In [39]:
# 모델 훈련
hist = model.fit(X_train, y_train,
                 batch_size=BATCH,
                 epochs=100,
                 validation_split=0.2)

Epoch 1/100
225/225 [==============================] - 1s 5ms/step - loss: -0.2860 - val_loss: 0.0059
Epoch 2/100
225/225 [==============================] - 1s 5ms/step - loss: -0.2516 - val_loss: 0.0200
Epoch 3/100
225/225 [==============================] - 1s 5ms/step - loss: -0.2798 - val_loss: 0.0123
Epoch 4/100
225/225 [==============================] - 1s 5ms/step - loss: -0.2730 - val_loss: 0.0333
Epoch 5/100
225/225 [==============================] - 1s 5ms/step - loss: -0.3806 - val_loss: 0.0547
Epoch 6/100
225/225 [==============================] - 1s 5ms/step - loss: -0.4116 - val_loss: 0.0041
Epoch 7/100
225/225 [==============================] - 1s 5ms/step - loss: -0.3870 - val_loss: 0.0043
Epoch 8/100
225/225 [==============================] - 1s 5ms/step - loss: -0.4077 - val_loss: 0.0047
Epoch 9/100
225/225 [==============================] - 1s 5ms/step - loss: -0.4762 - val_loss: 0.0210
Epoch 10/100
225/225 [==============================] - 1s 5ms/step - loss: -0.518

KeyboardInterrupt: ignored

# _8_. 결과 확인

In [ ]:
# # loss 시각화
# plt.plot(hist.history['loss'], label='Train Loss')
# plt.title('Loss Trajectory')
# plt.legend()
# plt.show()

In [ ]:
model.predict(X_test)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=2000)

preds = model.predict(X_test)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0
# score = compare preds and y_test

# 모델 예측 값 확인
y_pred = model.predict(X_test)

# 랜덤하게 예측된 라벨과 정답 확인
print ("이전 bigram","|Actual","|Predicted","\n")

NUM_DISPLAY = int(input('확인할 라벨 수 설정: '))
for i in random.sample(range(len(X_test_tg)), NUM_DISPLAY):
    print(i, X_test_tg[i], "|", idx2word[np.argmax(y_test[i])], "|", idx2word[np.argmax(y_pred[i])])

이전 bigram |Actual |Predicted 

확인할 라벨 수 설정: 10
486 better and | this | the
2821 lessons thought | alice | alice
6876 the further | off | only
870 little golden | key | key
3045 positively refused | to | tell
4520 change to | dull | tinkling
3935 little feeble | squeaking | door
119 alice was | very | beginning
502 puzzled he | must | said
5482 this seemed | to | quite


In [ ]:
# 

# ============ 테스트 ============

In [ ]:
content

'Alice\'s Adventures in Wonderland\n\n                ALICE\'S ADVENTURES IN WONDERLAND\n\n                          Lewis Carroll\n\n               THE MILLENNIUM FULCRUM EDITION 3.0\n\n\n\n\n                            CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought Alice `without pictures or conversation?\'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid), whether\nthe pleasure of making a daisy-chain would be worth the trouble\nof getting up and picking the daisies, when suddenly a White\nRabbit with pink eyes ran close by her.\n\n  There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to he

In [ ]:
new_text[0].split()[0:N-1]

['alice', 'adventures']

In [15]:
for i in range(5):
    print(X_trigram[i], " ", y_trigram[i])

alice adventures in   wonderland alice
adventures in wonderland   alice adventures
in wonderland alice   adventures in
wonderland alice adventures   in wonderland
alice adventures in   wonderland lewis


In [ ]:
print(vectorizer.fit_transform(X_trigram)) # 각각 위치 어디에 있는지 나타냄.

  (0, 51)	1
  (0, 27)	1
  (1, 27)	1
  (1, 1088)	1
  (2, 1088)	1
  (2, 2510)	1
  (3, 51)	1
  (3, 2510)	1
  (4, 51)	1
  (4, 27)	1
  (5, 27)	1
  (5, 1088)	1
  (6, 1088)	1
  (6, 2510)	1
  (7, 2510)	1
  (7, 1242)	1
  (8, 1242)	1
  (8, 303)	1
  (9, 303)	1
  (9, 2205)	1
  (10, 2205)	1
  (10, 1363)	1
  (11, 1363)	1
  (11, 869)	1
  (12, 869)	1
  :	:
  (25626, 1975)	1
  (25627, 1975)	1
  (25627, 1138)	1
  (25628, 1138)	1
  (25628, 1775)	1
  (25629, 1010)	1
  (25629, 1775)	1
  (25630, 1010)	1
  (25630, 1526)	1
  (25631, 1526)	1
  (25631, 347)	1
  (25632, 347)	1
  (25632, 1245)	1
  (25633, 70)	1
  (25633, 1245)	1
  (25634, 2205)	1
  (25634, 70)	1
  (25635, 2205)	1
  (25635, 970)	1
  (25636, 2146)	1
  (25636, 970)	1
  (25637, 507)	1
  (25637, 2146)	1
  (25638, 2205)	1
  (25638, 507)	1


In [33]:
print(vectorizer.fit_transform(y_trigram)) # 각각 위치 어디에 있는지 나타냄.

  (0, 2510)	1
  (0, 51)	1
  (1, 51)	1
  (1, 27)	1
  (2, 27)	1
  (2, 1088)	1
  (3, 2510)	1
  (3, 1088)	1
  (4, 2510)	1
  (4, 1242)	1
  (5, 1242)	1
  (5, 303)	1
  (6, 303)	1
  (6, 2205)	1
  (7, 2205)	1
  (7, 1363)	1
  (8, 1363)	1
  (8, 869)	1
  (9, 869)	1
  (9, 641)	1
  (10, 641)	1
  (10, 334)	1
  (11, 334)	1
  (11, 591)	1
  (12, 2205)	1
  :	:
  (25624, 1138)	1
  (25625, 1138)	1
  (25625, 1775)	1
  (25626, 1010)	1
  (25626, 1775)	1
  (25627, 1010)	1
  (25627, 1526)	1
  (25628, 1526)	1
  (25628, 347)	1
  (25629, 347)	1
  (25629, 1245)	1
  (25630, 70)	1
  (25630, 1245)	1
  (25631, 2205)	1
  (25631, 70)	1
  (25632, 2205)	1
  (25632, 970)	1
  (25633, 2146)	1
  (25633, 970)	1
  (25634, 507)	1
  (25634, 2146)	1
  (25635, 2205)	1
  (25635, 507)	1
  (25636, 2205)	1
  (25636, 664)	1


In [ ]:
np.argmax(np.array(vectorizer.fit_transform(X_trigram).todense())[0]) # 27번째 있는 단어

27

In [ ]:
np.argmax(np.array(vectorizer.fit_transform(X_trigram).todense())[1])

27

In [ ]:
np.argmax(np.array(vectorizer.fit_transform(X_trigram).todense())[2])

1088